# RAG System Testing Notebook

This notebook demonstrates how to test the RAG system with various medical queries.

## Overview
- Test API endpoints
- Test disease prediction
- Evaluate response quality
- Performance analysis
- End-to-end testing


In [ ]:
# Import required libraries
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from pathlib import Path
from typing import List, Dict, Any

# Set up plotting
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# API configuration
BASE_URL = "http://localhost:8000"
print(f"Testing API at: {BASE_URL}")


## 1. Test API Health and Endpoints


In [ ]:
# Test health endpoint
try:
    response = requests.get(f"{BASE_URL}/health", timeout=5)
    print("✅ Health check successful:")
    print(f"   Status: {response.status_code}")
    print(f"   Response: {response.json()}")
except requests.exceptions.RequestException as e:
    print(f"❌ Health check failed: {e}")

# Test info endpoint
try:
    response = requests.get(f"{BASE_URL}/info", timeout=5)
    print("\n✅ Info endpoint successful:")
    print(f"   Status: {response.status_code}")
    print(f"   Response: {response.json()}")
except requests.exceptions.RequestException as e:
    print(f"❌ Info endpoint failed: {e}")

# Test symptoms endpoint
try:
    response = requests.get(f"{BASE_URL}/symptoms", timeout=5)
    print("\n✅ Symptoms endpoint successful:")
    print(f"   Status: {response.status_code}")
    symptoms = response.json()['symptoms']
    print(f"   Available symptoms: {len(symptoms)}")
    print(f"   Sample symptoms: {symptoms[:5]}")
except requests.exceptions.RequestException as e:
    print(f"❌ Symptoms endpoint failed: {e}")


## 2. Test Disease Prediction


In [ ]:
# Test cases for disease prediction
test_cases = [
    {
        "name": "Common Cold Symptoms",
        "symptoms": ["fever", "cough", "headache", "runny_nose"]
    },
    {
        "name": "Skin Condition",
        "symptoms": ["itching", "skin_rash", "nodal_skin_eruptions"]
    },
    {
        "name": "Respiratory Issue",
        "symptoms": ["chills", "fever", "sweating", "cough"]
    },
    {
        "name": "Digestive Problem",
        "symptoms": ["nausea", "vomiting", "diarrhea", "abdominal_pain"]
    },
    {
        "name": "Neurological Symptoms",
        "symptoms": ["headache", "dizziness", "blurred_vision"]
    }
]

# Test each case
results = []
for i, test_case in enumerate(test_cases):
    print(f"\n🧪 Test Case {i+1}: {test_case['name']}")
    print(f"   Symptoms: {test_case['symptoms']}")
    
    try:
        start_time = time.time()
        response = requests.post(
            f"{BASE_URL}/predict-disease",
            json={"symptoms": test_case['symptoms']},
            timeout=10
        )
        end_time = time.time()
        
        if response.status_code == 200:
            result = response.json()
            print(f"   ✅ Success ({(end_time - start_time)*1000:.1f}ms)")
            print(f"   Predictions: {[p['disease'] for p in result['predictions']]}")
            print(f"   Confidence: {[f\"{p['confidence']:.3f}\" for p in result['predictions']]}")
            
            results.append({
                'test_case': test_case['name'],
                'symptoms': test_case['symptoms'],
                'predictions': result['predictions'],
                'response_time': end_time - start_time,
                'success': True
            })
        else:
            print(f"   ❌ Failed with status {response.status_code}")
            results.append({
                'test_case': test_case['name'],
                'symptoms': test_case['symptoms'],
                'predictions': [],
                'response_time': end_time - start_time,
                'success': False
            })
    except requests.exceptions.RequestException as e:
        print(f"   ❌ Request failed: {e}")
        results.append({
            'test_case': test_case['name'],
            'symptoms': test_case['symptoms'],
            'predictions': [],
            'response_time': 0,
            'success': False
        })

print(f"\n📊 Test Summary:")
print(f"   Total tests: {len(test_cases)}")
print(f"   Successful: {sum(1 for r in results if r['success'])}")
print(f"   Failed: {sum(1 for r in results if not r['success'])}")
print(f"   Average response time: {np.mean([r['response_time'] for r in results if r['success']]):.3f}s")


## 3. Performance Analysis


In [ ]:
# Analyze performance metrics
successful_results = [r for r in results if r['success']]

if successful_results:
    # Response time analysis
    response_times = [r['response_time'] for r in successful_results]
    
    plt.figure(figsize=(15, 5))
    
    # Response time distribution
    plt.subplot(1, 3, 1)
    plt.hist(response_times, bins=10, alpha=0.7, color='skyblue')
    plt.title('Response Time Distribution')
    plt.xlabel('Response Time (seconds)')
    plt.ylabel('Frequency')
    
    # Confidence scores
    plt.subplot(1, 3, 2)
    all_confidences = []
    for result in successful_results:
        for pred in result['predictions']:
            all_confidences.append(pred['confidence'])
    
    if all_confidences:
        plt.hist(all_confidences, bins=15, alpha=0.7, color='lightgreen')
        plt.title('Confidence Score Distribution')
        plt.xlabel('Confidence Score')
        plt.ylabel('Frequency')
    
    # Success rate
    plt.subplot(1, 3, 3)
    success_count = len(successful_results)
    failure_count = len(results) - success_count
    plt.pie([success_count, failure_count], labels=['Success', 'Failure'], 
            autopct='%1.1f%%', colors=['lightgreen', 'lightcoral'])
    plt.title('Test Success Rate')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("📈 Performance Statistics:")
    print(f"   Average response time: {np.mean(response_times):.3f}s")
    print(f"   Min response time: {np.min(response_times):.3f}s")
    print(f"   Max response time: {np.max(response_times):.3f}s")
    print(f"   Success rate: {success_count/len(results)*100:.1f}%")
    
    if all_confidences:
        print(f"   Average confidence: {np.mean(all_confidences):.3f}")
        print(f"   Min confidence: {np.min(all_confidences):.3f}")
        print(f"   Max confidence: {np.max(all_confidences):.3f}")
else:
    print("❌ No successful tests to analyze")


## 4. Test Disease Details Endpoint


In [ ]:
# Test disease details endpoint
diseases_to_test = ["Common Cold", "Fungal infection", "Malaria", "Diabetes", "Hypertension"]

print("🔍 Testing Disease Details Endpoint:")
print("=" * 50)

for disease in diseases_to_test:
    try:
        response = requests.get(f"{BASE_URL}/disease/{disease}", timeout=5)
        
        if response.status_code == 200:
            details = response.json()
            print(f"\n✅ {disease}:")
            print(f"   Description: {details.get('description', 'N/A')[:100]}...")
            print(f"   Precautions: {details.get('precautions', [])}")
        elif response.status_code == 404:
            print(f"\n❌ {disease}: Not found")
        else:
            print(f"\n⚠️ {disease}: Status {response.status_code}")
            
    except requests.exceptions.RequestException as e:
        print(f"\n❌ {disease}: Request failed - {e}")

print("\n" + "=" * 50)
print("Disease details testing completed!")


## 5. Save Test Results


In [ ]:
# Save test results to file
results_dir = Path("../data/evaluation")
results_dir.mkdir(parents=True, exist_ok=True)

# Create summary report
test_summary = {
    "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
    "total_tests": len(results),
    "successful_tests": len(successful_results),
    "failed_tests": len(results) - len(successful_results),
    "success_rate": len(successful_results) / len(results) * 100 if results else 0,
    "average_response_time": np.mean([r['response_time'] for r in successful_results]) if successful_results else 0,
    "test_results": results
}

# Save to JSON
with open(results_dir / "rag_test_results.json", "w") as f:
    json.dump(test_summary, f, indent=2)

print("💾 Test results saved to data/evaluation/rag_test_results.json")

# Create CSV report for analysis
if successful_results:
    csv_data = []
    for result in successful_results:
        for pred in result['predictions']:
            csv_data.append({
                'test_case': result['test_case'],
                'symptoms': ', '.join(result['symptoms']),
                'predicted_disease': pred['disease'],
                'confidence': pred['confidence'],
                'response_time': result['response_time']
            })
    
    df_results = pd.DataFrame(csv_data)
    df_results.to_csv(results_dir / "rag_test_results.csv", index=False)
    print("📊 Detailed results saved to data/evaluation/rag_test_results.csv")
    
    print(f"\n📋 Final Summary:")
    print(f"   Total test cases: {len(results)}")
    print(f"   Successful predictions: {len(successful_results)}")
    print(f"   Success rate: {test_summary['success_rate']:.1f}%")
    print(f"   Average response time: {test_summary['average_response_time']:.3f}s")
else:
    print("❌ No successful tests to save")
